In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

In [3]:
df = pd.read_csv('disaster_tweets_data.csv',encoding='latin1')
df.head()

,tweets,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
df = df.iloc[:,:2]
df.head()

,tweets,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
df.columns = ['tweets','target']
df.head()

,tweets,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
df['tweets'][11]

'Haha South Tampa is getting flooded hah- WAIT A SECOND I LIVE IN SOUTH TAMPA WHAT AM I GONNA DO WHAT AM I GONNA DO FVCK #flooding'

In [7]:
lm = WordNetLemmatizer()
sw = stopwords.words('English')
print(sw[:5])


['i', 'me', 'my', 'myself', 'we']


In [8]:
data = []
for i in df['tweets'].iloc[:5]:
    t = i.lower()                     # lower case conversion
    t = re.sub('[^A-Za-z0-9]',' ',t)  # removing punc
    t = word_tokenize(t)              # word tekenize
    t = [i for i in t if i not in sw] # sw removal
    t = [lm.lemmatize(i) for i in t]  # lemmatization
    t = " ".join(t)                   # joinig words to froms sentences
    data.append(t)
print(data)

['deed reason earthquake may allah forgive u', 'forest fire near la ronge sask canada', 'resident asked shelter place notified officer evacuation shelter place order expected', '13 000 people receive wildfire evacuation order california', 'got sent photo ruby alaska smoke wildfire pours school']


In [9]:
data = []
for i in df['tweets']:
    t = i.lower()                     # lower case conversion
    t = re.sub('[^A-Za-z0-9]',' ',t)  # removing punc
    t = word_tokenize(t)              # word tekenize
    t = [i for i in t if i not in sw] # sw removal
    t = [lm.lemmatize(i) for i in t]  # lemmatization
    t = " ".join(t)                   # joinig words to froms sentences
    data.append(t)
print(len(data))

7613


In [10]:
x = np.array(data)
y = df['target']
print(x.shape)
print(y.shape)

(7613,)
(7613,)


In [11]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.30,random_state=42)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(5329,)
(2284,)
(5329,)
(2284,)


In [12]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1500)

x_train_cv = cv.fit_transform(x_train)
x_test_cv = cv.transform(x_test)
print(x_train_cv.shape)
print(x_test_cv.shape)

(5329, 1500)
(2284, 1500)


#### ML model building

In [13]:
from sklearn.naive_bayes import MultinomialNB

In [14]:
mnb = MultinomialNB()
mnb.fit(x_train_cv,y_train)

MultinomialNB()

In [15]:
print('Train Score',mnb.score(x_train_cv,y_train))  # Train Acc
print('Test Score',mnb.score(x_test_cv,y_test))     # Test Acc

Train Score 0.8239819853631075
Test Score 0.7841506129597198


In [16]:
ypred_mnb = mnb.predict(x_test_cv)

In [17]:
from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(y_test,ypred_mnb))
print(classification_report(y_test,ypred_mnb))

[[1113  205]
 [ 288  678]]
              precision    recall  f1-score   support

           0       0.79      0.84      0.82      1318
           1       0.77      0.70      0.73       966

    accuracy                           0.78      2284
   macro avg       0.78      0.77      0.78      2284
weighted avg       0.78      0.78      0.78      2284

